# Create ELT aperture files

Need:
- aperture
- indexed aperture containing segment positions
- Lyot stop (for now empty)
- apodizer (for now empty)

In [ ]:
import os
from astropy.io import fits
import hcipy
import matplotlib.pyplot as plt
import numpy as np

## Telescope aperture

In [ ]:
pupil_grid = hcipy.make_pupil_grid(1000, diameter=40)

# Plain aperture
aper_gen = hcipy.make_elt_aperture(normalized=False)
aper = hcipy.evaluate_supersampled(aper_gen, pupil_grid, 4)

In [ ]:
# Indexed aperture
aper_ind_gen, seg_gen = hcipy.make_elt_aperture(normalized=False, segment_transmissions=np.arange(798)+1,
                                                return_segments=True)
aper_ind = hcipy.evaluate_supersampled(aper_ind_gen, pupil_grid, 4)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
hcipy.imshow_field(aper, cmap='Greys_r', origin='lower')
plt.colorbar()
plt.subplot(1,2,2)
hcipy.imshow_field(aper_ind, origin='lower')
plt.colorbar()

In [ ]:
type(seg_gen[0])

In [ ]:
this = seg_gen.points

In [ ]:
hdu_indexed = fits.PrimaryHDU()
hdr_indexed = hdu_indexed.header.copy()

In [ ]:
seg_xys = seg_gen.points
for segment in range(seg_xys.shape[0]):
    print('seg' + str(segment + 1) + '_x', seg_gen.x[segment], 'x-position segment ' + str(segment + 1))
    print('seg' + str(segment + 1) + '_y', seg_gen.y[segment], 'y-position segment ' + str(segment + 1))
    hdr_indexed.set('seg' + str(segment + 1) + '_x', seg_gen.x[segment], 'x-position segment ' + str(segment + 1))
    hdr_indexed.set('seg' + str(segment + 1) + '_y', seg_gen.y[segment], 'y-position segment ' + str(segment + 1))

In [ ]:
fits.writeto('ELT_aperture.fits', aper.shaped, overwrite=True)
fits.writeto('ELT_aperture_indexed.fits', aper_ind.shaped, hdr_indexed, overwrite=True)

In [ ]:
apodizer = np.ones_like(aper)
lyot_stop = np.ones_like(aper)

In [ ]:
fits.writeto('spc_apodizer.fits', apodizer.shaped, overwrite=True)
fits.writeto('lyot_stop.fits', lyot_stop.shaped, overwrite=True)